# Приложение: сравнение различных методов оценки А/Б-тестов

*Приведен обзор методов для анализа А/Б-тестов*.

* [Введение](#Введение)
* [Оценки среднего и дисперсии на основе сэмпла](#Оценки-среднего-и-дисперсии-на-основе-сэмпла)
* [Бутстрап](#Бутстрап)
* [Метод максимального правдоподобия](#Метод-максимального-правдоподобия)
* Проверка статистических гипотез
* [Байесовское моделирование](#Байесовское-моделирование)
* [Количество правильно угаданных вариантов](#Количество-правильно-угаданных-вариантов)

# Введение

В ч.1 обсуждался байесовским подход к оценке А/Б-тестов.  
Было показано, как в рамках этого подхода ответить на вопросы
- Какой вариант лучше и насколько?
- Каковы оценки целевой метрики в каждом варианте?
- Насколько уверены в оценке?
- Сколько должен продолжаться эксперимент?

Полезно сравнить байесовское моделирование с другими подходами.  

## Сравнение средних в сэмплах

Закон больших чисел  
https://en.wikipedia.org/wiki/Law_of_large_numbers  

Среднее в сэмпле - состоятельная и несмещенная оценка среднего в распределении  
https://en.wikipedia.org/wiki/Estimator

Среднее в выборке сходится к среднему в распределении.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#todo: update scipy; make venv

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.6

n = np.arange(10, 5000, 100)
k_a = stats.binom.rvs(n, p_a_exact)
k_b = stats.binom.rvs(n, p_b_exact)
p_a_samp = k_a/n
p_b_samp = k_b/n

fig = go.Figure()
fig.add_trace(go.Scatter(x=n, y=p_a_samp, name='A: p sample'))
fig.add_trace(go.Scatter(x=[0, np.max(n)], y=[p_a_exact, p_a_exact],
                         mode='lines',
                         line_dash='dash', line_color='black', name='A: p exact'))
fig.add_trace(go.Scatter(x=n, y=p_b_samp, name='B: p sample'))
fig.add_trace(go.Scatter(x=[0, np.max(n)], y=[p_b_exact, p_b_exact],
                         mode='lines',
                         line_dash='dash', line_color='black', name='B: p exact'))
fig.update_layout(
    title='Exact and Sample Means for Binomial Distribution',
    xaxis_title='Sample Size',
    yaxis_title='p',
    yaxis_range=[0, 1],
    xaxis_range=[0, np.max(n)],
    height=550
)
fig.show()

Позволяет получить точечные оценки.  
Можно выбрать группу с большим средним.  
Но остается вопрос, с какой вероятностью одна группа лучше другой.  

In [ ]:
def compare_groups_binomial(sa, na, sb, nb):
    pa = sa / na
    pb = sb / nb
    if pa > pb:
        res = 'A'
    elif pa < pb:
        res = 'B'
    else:
        res = None
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.75

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by means comparison: {compare_groups_binomial(sa, n, sb, n)}')

|Метод |Оценки целевой метрики | Какая группа лучше | Насколько лучше | Сколько продолжать эксперимент|  
|----|---|---|----|----|
|Сравнение средних| |

In [ ]:
df_methods = pd.DataFrame(columns=[
    'Метод', 
    'Оценки целевой метрики', 
    'Критерий выбора группы', 
    'Оценка длительности'
])
df_methods = pd.concat([df_methods, pd.DataFrame([{
    'Метод': 'Сравнение средних', 
    'Оценки целевой метрики': 'Точечные оценки средних', 
    'Критерий выбора группы': 'Где "лучше" среднее', 
    'Оценка длительности': 'Не рассчитывается; можно договориться о "достаточно большой" выборке'
}])], ignore_index=True)
df_methods

## Стандартная ошибка среднего и интервал с помощью неравенства Чебышева

* Посчитать средние и дисперсии в каждой группе
* Посчитать стандартные ошибки средних
* С помощью неравенства Чебышева оценить интервал возможных значений
* Построить разность средних и дисперсию разности средних
* Посчитать, с какой вероятностью можно получить 0 или более экстремальное значение.

Можно построить стандартную ошибку среднего.   
https://en.wikipedia.org/wiki/Standard_error#Derivation

$$
D[{\bar{x}}] = \frac{\sigma^2}{n}
$$

Оценку дисперсии также можно построить по сэмлу.  

С помощью неравенства Чебышева можно оценить вероятность отклонения случайной величины от среднего.
Часто используют для оценки отклонения на несколько стандартных отклонений.    
https://en.wikipedia.org/wiki/Chebyshev%27s_inequality

$$
P(|X - \mu| \ge k \sigma) \le \frac{1}{k^2}
$$

При $k=5$

$$
P(|X - \mu| \ge 5 \sigma) \le \frac{1}{25} = 0.04
$$

Одностороннее неравенство Чебышева:  
https://en.wikipedia.org/wiki/Chebyshev%27s_inequality#Cantelli's_inequality

Для дисперсии нужны поправки к значению в сэмпле чтобы получить несмещенную оценку  
https://en.wikipedia.org/wiki/Unbiased_estimation_of_standard_deviation  

Среднее разности случайных величин:  
https://en.wikipedia.org/wiki/Expected_value#Properties

$$
E[X - Y] = E[X] - E[Y]
$$

Дисперсия разности случайных величин:  
https://en.wikipedia.org/wiki/Standard_deviation#Identities_and_mathematical_properties  
https://en.wikipedia.org/wiki/Covariance

$$
D[X - Y] = D[X] + D[Y] - 2cov[X,Y]
$$

Формально величины X и Y ожидаются независимыми.  
Если отключить одну группу, на другую это не повлияет.  
Для независимых величин $cov[X,Y] = 0$.  

На практике можно оставить это слагаемое.

(для конверсий $\tilde{p}_A$ похоже на биномиальное распределение, но отнесенное к n)

$$
A = [1, 0, 1, 0, 0, ... , 1]
\\
s_A, n_A
\\
E_A \equiv p_A = s_A / n_A
\\
D_A = p_A (1 - p_A)
\\
\tilde{p}_A = \frac{1}{n} \left( \tilde{A} + \tilde{A} + \dots + \tilde{A} \right)
\\
E[\tilde{p}_A] = E_A = p_A
\\
D[\tilde{p}_A] = \frac{D_A}{n_A}
\\
\Delta = \tilde{p}_A - \tilde{p}_B
\\
E_{\Delta} = E[\tilde{p}_A - \tilde{p}_B] = E[\tilde{p}_A] - E[\tilde{p}_B] = p_A - p_B
\\
D_{\Delta} = D[\tilde{p}_A - \tilde{p}_B] = D[\tilde{p}_A] + D[\tilde{p}_B]
\\
\sigma_{\Delta} = \sqrt{D[p_A] + D[p_B]}
\\
P(|0 - E_{\Delta}| \ge k \sigma_{\Delta}) \le \frac{1}{k^2}
\\
P \left( \frac{|E_{\Delta}|}{\sigma_{\Delta}} \ge k \right) \le \frac{1}{k^2}
$$

вопрос интерпретации вероятности.  
и доверительных интервалов вокруг средних значений.  
вводится новая случайная величина - сумма N наблюдений.  
делаются утверждения о ее распределении.  
как это соотносится с исходными вопросами?

Стоит обратить внимание, что приведенные методы не делают предположений о форме распределений тестируемых величин. Единственное, что требуется - существование конечных средних и дисперсий. Чаще всего эти предположения можно считать выполненными. 

In [ ]:
def std_err_mean_binom(s, n):
    p = s / n
    return np.sqrt(p * (1 - p) / n)

def compare_groups_binomial(sa, na, sb, nb):
    pa = sa / na
    pb = sb / nb
    std_err_mean_a = std_err_mean_binom(sa, na)
    std_err_mean_b = std_err_mean_binom(sb, nb)
    mean_diff = pb - pa
    std_err_mean_diff = np.sqrt(std_err_mean_a**2 + std_err_mean_b**2)
    k = np.abs(mean_diff) / std_err_mean_diff
    if pa > pb and k >= 5:
        res = 'A'
    elif pa < pb and k >= 5:
        res = 'B'
    else:
        res = None
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.75

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by means comparison: {compare_groups_binomial(sa, n, sb, n)}')

## Оценки разности средних с учетом центральной предельной теоремы

# Бутстрап

Еще один вариант закона больших чисел говорит, что по мере набора данных частоты возможных значений в сэмле будут приближаться к их вероятности. Это является основой для частотной интерпретации вероятности.

Также на этом основании имеющийся сэмпл данных можно считать приближением к точному распределению.

Общее название - методы повторной выборки данных или ресэмплинг.  
Один из методов - метод складного ножа (Jackknife).   
Еще один популярный метод - бутстрап.  
Далее обсуждается только последний.

Бутстрап:  
https://projecteuclid.org/journals/annals-of-statistics/volume-7/issue-1/Bootstrap-Methods-Another-Look-at-the-Jackknife/10.1214/aos/1176344552.full

*What Teachers Should Know about the Bootstrap: Resampling in the Undergraduate Statistics Curriculum*  
https://arxiv.org/abs/1411.5279  
https://arxiv.org/pdf/1411.5279.pdf  

Можно оценить неопределенность средних значений.

In [ ]:
def bootstrap_means_rawdata(dt, k_bootstrap):
    bs = np.random.choice(dt, size=(k_bootstrap, n), replace=True)
    means = bs.mean(axis=1)
    return means

def bootstrap_means_binomial(s, n, k_bootstrap):
    p = s/n
    bs = stats.binom.rvs(n, p, size=k_bootstrap)
    means = bs / n
    return means

In [ ]:
p_exact_a = 0.7
p_exact_b = 0.65

n = 5000
s_a = stats.binom.rvs(n, p_exact_a)
s_b = stats.binom.rvs(n, p_exact_b)
p_sample_a = s_a/n
p_sample_b = s_b/n

k_bootstrap = 30000
means_a = bootstrap_means_binomial(s_a, n, k_bootstrap)
means_b = bootstrap_means_binomial(s_b, n, k_bootstrap)

fig = go.Figure()
fig.add_trace(go.Histogram(x=means_a, histnorm='percent',
                           name='A',
                           opacity=0.3))
#fig.add_vline(x=p_sample_a, line_dash='dash')
fig.add_vline(x=p_exact_a)
fig.add_trace(go.Histogram(x=means_b, histnorm='percent',
                           name='B',
                           opacity=0.3))
#fig.add_vline(x=p_sample_b, line_dash='dash')
fig.add_vline(x=p_exact_b)
# fig.add_trace(go.Scatter(x=[p_exact_b, p_exact_b], y=[0, 1], 
#                          mode='lines', line_color='black', line_dash='dash',
#                          name='Exact Mean B'))
fig.update_layout(title='Means Bootstrap Dists',
                  xaxis_title='p',
                  yaxis_title='Prob',
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

In [ ]:
def compare_groups_bootstrap_binomial(sa, na, sb, nb):
    k_bootstrap = 30000
    means_a = bootstrap_means_binomial(sa, na, k_bootstrap)
    means_b = bootstrap_means_binomial(sb, nb, k_bootstrap)
    p_ea_gt_eb = np.sum(means_a > means_b) / len(means_a)
    p_level = 0.95
    res = None
    if p_ea_gt_eb >= p_level:
        res = 'A'
    elif (1 - p_ea_gt_eb) >= p_level:
        res = 'B'
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.67

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by bootstrap comparison: {compare_groups_bootstrap_binomial(sa, n, sb, n)}')

Вычислительно может быть затратно.  
Есть пара трюков для ускорения вычислений.  

Пуассоновский бутстрап:  
https://www.unofficialgoogledatascience.com/2015/08/an-introduction-to-poisson-bootstrap26.html  

Сэмплировать из выборки [x1, x2, ..., xn] с повторением все равно, что генерировать набор
мультиномиальных коэффициетов x ~ Multinomial(n, 1/n). Для больших n предлагается заменить Multinomial(n, 1/n) на n сэмплов из биномиального распределение Binom(n, 1/n). Количество точек в каждом векторе при этом перестает быть одинаковым. Еще один шаг - заменить биномиальное распределение Binom(n, 1/n) на распределение Пуассона Poisson(1). Сэмплировать из него. 

Байесовский бутстрап:  
https://gdmarmerola.github.io/the-bayesian-bootstrap/ - примеры  
https://projecteuclid.org/journalArticle/Download?urlId=10.1214%2Faos%2F1176345338 - оригинальный текст  
https://www.sumsar.net/blog/2015/04/the-non-parametric-bootstrap-as-a-bayesian-model/  

Есть данные x_1, ..., x_n.  
Сгенерировать (n-1) точку u_1, ..., u_{n-1} из равномерного распределениея U(0,1);
Отсортировать их;  
Дополнить точками [0, ... , 1]  
Посчитать разности g_i = u_i - u_i-1, i>=1.  
Использовать g_i как вероятности для [x1, ..., x_n].   
Среднее  
m = sum g_i x_i

Есть отличия в интерпретации от обычного бутстрапа.

Ограничения?  
Бутстрап несколько хуже работает для непрерывных величин.  
При малых размерах исходных данных.  
Если часть данных мало представлена (исходное распределение скошено, в выборке мало значений из хвоста).  

# Метод максимального правдоподобия

Предполагается аналитическое распределение для данных - (функция правдоподобия).  
Значения параметров выбирается такие, которые будут максимизировать эту функцию при имеющихся данных.  

https://en.wikipedia.org/wiki/Maximum_likelihood_estimation#Asymptotics  

Для некоторых моделей можно проверси аналитические вычисления.  
Для биномиального распределения максимум правдоподобия совпадет со средним в сэмпле.  
Для нормального - со средним и дисперсией в сэмле. 

В байесовском подходе параметры интерпретируются как случайные величины с распределением.  
В методе максимального правдоподобия - считаются неизвестной постоянной величиной.

Оценка параметров этим методом совпадет с максимум апостериорного распределения, если априорное распределение равномерное

https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation  

Выбрана модель и оценены параметры, дальше можно сравнивать эти распределения.  
Если есть возможность посчитать средние по параметрам аналитически, то можно выбрать группу с большим средним.  
Метод формально говорит о полной уверенности.

Поскольку вместо распределения параметров используется точечная оценка, метод будет давать завышенную уверенность по сравнению со сравнением апостериорных распределений в байесовском подходе.  

**Доверительные интервалы** 

# Проверка статистических гипотез

Для анализа А/Б-тестов иногда используют метод статистических гипотез. 
Общая идея - предположить, что между группами нет разницы, после чего посчитать, насколько такое предположение объясняет экспериментальные данные. Если вероятность получить данные мала, то считается, что предположение можно отвергнуть, т.е. между группами есть значимая разница.  

Для вычисления вероятности получить данные в рамках предположения об эквивалентности групп существует многообразие статистических тестов.

### Минимальный пример проверки статистической гипотезы

Есть экспериментальные данные $data$.  
Есть гипотеза $H$ об этих данных или их свойствах.  
Есть случайная величина $Test$, распределение которой $P_{Test|H}(x)$ в предположении $H$ известно.  
По фактическим данным считается "тестовая статистика"  $x_{fact}$ .   
Считаем вероятность получить "фактическое или более экстремальное" значение "тестовой статистики" $p = P_{Test|H}(x \ge x_{fact} )$ или $p = P_{Test|H}(x \le x_{fact} )$ . Т.е. $p = CDF_{Test|H}(x_{fact})$ или $p = 1 - CDF_{Test|H}(x_{fact})$ .  
Если вероятность мала, то гипотезу $H$ считают неверной.

Пример:
- данные: в 1000 бросках монетки в 700 случаях выпал орел.  
- гипотеза: вероятность выпадения орла 0.5   
- случайная величина с известным распределением: вероятность k успехов в N попытках при вероятности $p$ успеха в одной попытке задается биномиальным распределением $Binom(p; N, k)$  
- тестовая статистика: число успехов k=700 в общих попытках N=1000
- p-значение: вероятность получить 700 или больше успехов в 1000 попытках $\sum_{i \ge 700}Binom(0.5; 1000, i)$

Полезно сравнить с байесовским подходом.  
Данные те же.  
Случайная величина с известным распределением - функция правдоподобия.  
Гипотеза - конкретное значение параметров.  
Тестовая статистика - аргумент в функции правдоподобия x_fact = f(data).   
p-значение: sum x L(x >= x_fact | H)  
Аналога априорного распределения нет.  
Иначе устроена интерпретация: в проверке стат. гипотез решение принимают по функции правдоподобия, в байесовском подходе пересчитывают в вероятность.  

Сложность с интерпретацией $p$-значения.  

Для оценки параметров проверка статистических гипотез обычно не используется.   
Используется метод максимального правдоподобия или другие предположения.  

Для оценки интервалов параметров используются "доверительные интервалы".  
Формально неопределенности в параметрах нет.  
Сложности с интерпретацией доверительных интервалов:  


Все давно сделано:  
https://michael-franke.github.io/BDACM_2018/scripts/01_estimation_pValues.pdf

Тест Колмогорова-Смирнова  
https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test  

U-тест  
https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test  

### t-тест для средних

https://en.wikipedia.org/wiki/Student's_t-test

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.67

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

p_a = sa/n
a_var = n * p_a * (1 - p_a)
a_stderr = np.sqrt(a_var) / n

p_b = sb/n
b_var = n * p_b * (1 - p_b)
b_stderr = np.sqrt(b_var) / n

x=np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, p_a, a_stderr),
                         name='A'))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, p_b, b_stderr),
                         name='B'))
fig.update_layout(
    title='Means Normal Dists for t-test'
)
fig.show()

In [ ]:
def ttest_means_rawdata(a, b):
    t, p = stats.ttest_indep(a, b)
    return p
    

def compare_groups_ttest_binomial(sa, na, sb, nb):
    a_mean = sa/n
    a_var = n * a_mean * (1 - a_mean)
    a_stderr = np.sqrt(a_var) / n
    b_mean = sb/n
    b_var = n * b_mean * (1 - b_mean)
    b_stderr = np.sqrt(b_var) / n
    diff = a_mean - b_mean
    diff_stderr = np.sqrt(a_stderr**2 + b_stderr**2)
    pval = stats.norm.cdf(0, diff, diff_stderr)
    p_level = 0.95
    significant = (pval >= p_level) or (1 - pval) >= p_level
    if not significant:
        res = None
    elif significant and a_mean > b_mean:
        res = 'A'
    elif significant and b_mean > a_mean:
        res = 'B'
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.67

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by bootstrap comparison: {compare_groups_ttest_binomial(sa, n, sb, n)}')

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
#https://en.wikipedia.org/wiki/Welch%27s_t-test

a = np.zeros(n)
a[:sa] = 1
#print(sa)
#np.unique(a, return_counts=True)

b = np.zeros(n)
b[:sb] = 1

t, p = stats.ttest_ind(a, b, equal_var=False)#, alternative='less')
print(t,p)

p_a = sa/n
a_var = n * p_a * (1 - p_a)
a_stderr = np.sqrt(a_var) / n
p_b = sb/n
b_var = n * p_b * (1 - p_b)
b_stderr = np.sqrt(b_var) / n
diff = p_a - p_b
diff_stderr = np.sqrt(a_stderr**2 + b_stderr**2)
pval = stats.norm.cdf(0, diff, diff_stderr)
pval

### Перестановочный тест

https://en.wikipedia.org/wiki/Permutation_test  

Есть данные из двух групп.
Считаем среднее в каждой группе.
Считаем разницу.

Далее вопрос: насколько вероятно получить такую разницу если группы одинаковы?

Для ответа предполагаем, что группы значения пришли из общего распределения.
Из общего распределения сэмплим n_a значений, считаем среднее, потом n_b, также считаем среднее, считаем разницу между средними. 
Это повторяется по всем возможным комбинациям или пока не надоест. 
Строится распределение разностей средних.  
Определяется, где на этом распределении фактическое значение.  
Если оно "достаточно экстремально", группы объявляются разными.  

К перестановочным тестам применяют механику $p$-значений из проверки статистических гипотез (см. далее).  

In [ ]:
a_mean = 3
b_mean = 3.1
sigma = 1
samp_a = stats.norm.rvs(loc=a_mean, scale=sigma, size=30)
samp_b = stats.norm.rvs(loc=b_mean, scale=sigma, size=30)

x = np.linspace(0, 5, 1000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x=x, loc=a_mean, scale=sigma)))
fig.add_trace(go.Histogram(x=samp_a))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x=x, loc=b_mean, scale=sigma)))
fig.add_trace(go.Histogram(x=samp_b))
fig.show()

In [ ]:
exp_info = pd.DataFrame([
    {'group':'A', 'a':1, 'b':1, 'sample_size':100},
    {'group':'B', 'a':1, 'b':1.25, 'sample_size':100}])
exp_info.set_index('group', inplace=True)

exact_dist_a = stats.gamma(a=exp_info['a']['A'], scale=1/exp_info['b']['A'])
exact_dist_b = stats.gamma(a=exp_info['a']['B'], scale=1/exp_info['b']['B'])
exp_info['exact_mean'] = pd.Series({'A': exact_dist_a.mean(), 'B':exact_dist_b.mean()})
display(exp_info)

samp_a = exact_dist_a.rvs(size=exp_info['sample_size']['A'])
samp_b = exact_dist_b.rvs(size=exp_info['sample_size']['B'])

x = np.linspace(0, 10, 2000)
ymax = np.max([exact_dist_a.pdf(x), exact_dist_b.pdf(x)])

fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pdf(x), 
                         mode='lines', line_color=col,
                         name=f"Exact A: a={exp_info['a']['A']}, b={exp_info['b']['A']}"))
fig.add_trace(go.Histogram(x=samp_a, histnorm='probability density',
                           name='Sample A',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_a.mean(), exact_dist_a.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean A'))
col = 'blue'
fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pdf(x), 
                         mode='lines', line_color=col,
                         name=f"Exact B: a={exp_info['a']['B']}, b={exp_info['b']['B']}"))
fig.add_trace(go.Histogram(x=samp_b, histnorm='probability density',
                           name='Sample B',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_b.mean(), exact_dist_b.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean B'))
fig.update_layout(title='Exact Distributions and Samples',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

In [ ]:
pooled = np.concatenate([samp_a, samp_b])

n_resample = 10000
means_diff = np.empty(n_resample)

for i in range(n_resample):
    tmp = np.random.permutation(pooled)
    tmp_a = tmp[0:len(samp_a)]
    tmp_b = tmp[len(samp_a):]
    mean_diff = tmp_a.mean() - tmp_b.mean()
    means_diff[i] = mean_diff

#means_diff
diff_mean_fact = samp_a.mean() - samp_b.mean()

fig = go.Figure()
fig.add_trace(go.Histogram(x=means_diff, histnorm='probability density',
                           name='Means Diff',
                           opacity=0.3, marker_color=col))
fig.add_vline(x=diff_mean_fact)
fig.show()

pval = np.sum(means_diff[means_diff > diff_mean_fact]) / len(means_diff)
print(pval)

# Байесовское моделирование

Теория разобрана в предыдущих частях.

Основная идея - предположить аналитический вид распределения ("модель"). С помощью соотношения Байеса построить плотность вероятности параметров модели.  

In [ ]:
def posterior_binom_beta(sa, n, alpha=1, beta=1):
    alpha_post = alpha + sa
    beta_post = beta + (n - sa)
    return stats.beta(alpha_post, beta_post)

In [ ]:
pa_exact = 0.7
pb_exact = 0.65

n = 1000
sa = stats.binom.rvs(n, pa_exact)
sb = stats.binom.rvs(n, pb_exact)

post_dist_a = posterior_binom_beta(sa, n)
post_dist_b = posterior_binom_beta(sb, n)

x = np.linspace(0, 1, 1000)
ymax = np.max([post_dist_a.pdf(x), post_dist_b.pdf(x)])
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_dist_a.pdf(x), name='A'))
fig.add_trace(go.Scatter(x=[pa_exact, pa_exact], y=[0, ymax], 
                         mode='lines', line_dash='dash', line_color='black', 
                         name='A: p exact'))
fig.add_trace(go.Scatter(x=x, y=post_dist_b.pdf(x), name='B'))
fig.add_trace(go.Scatter(x=[pb_exact, pb_exact], y=[0, ymax],
                         mode='lines', line_dash='dash', line_color='black', 
                         name='B: p exact'))
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob',
                  xaxis_range=[0.5, 1],
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

In [ ]:
def compare_groups_bayes_binomial(sa, na, sb, nb):
    n_post_sample = 30000
    c_level = 0.95
    post_a = posterior_binom_beta(sa, na)
    post_b = posterior_binom_beta(sb, nb)
    post_samp_a = post_a.rvs(n_post_sample)
    post_samp_b = post_b.rvs(n_post_sample)
    pa_gt_pb = np.sum(post_samp_a > post_samp_b) / len(post_samp_a)
    res = None
    if pa_gt_pb >= c_level:
        res = 'A'
    elif pa_gt_pb <= (1 - c_level):
        res = 'B'
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.67

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by bayesian comparison: {compare_groups_bayes_binomial(sa, n, sb, n)}')

# Количество правильно угаданных вариантов

Две группы.  
Пусть сравниваются конверсии.  
Т.е. загадываются параметры двух пуассоновских процессов.  
Фиксируется количество наблюдений N.  
Для фиксированного количества наблюдений генерируется k пар параметров pa, pb.  
Для каждой пары генерируется число успехов ka, kb.  
Числа 

Генерация данных для эксперимента

In [ ]:
def gen_binomial_experiments(n_points, k_experiments, pa_base=0.1):
    pa = np.full(k_experiments, pa_base)
    pb = stats.uniform.rvs(loc=0.9*pa, scale=0.2*pa) #unif[loc, loc+scale]
    sa = stats.binom.rvs(n=n_points, p=pa, size=k_experiments)
    sb = stats.binom.rvs(n=n_points, p=pb, size=k_experiments)
    e = {
        'exp_type': 'binomial',
        'k_experiments': k_experiments,
        'n_points': n_points,
        'pa_exact': pa,
        'pb_exact': pb,
        'sa': sa,
        'sb': sb
    }
    return e

Сравнение экспериментов и подсчет количества правильно угаданных вариантов

In [ ]:
def compare_experiments_binomial_exact(exps):
    s = np.empty_like(exps['pa_exact'], dtype=object)
    s[exps['pa_exact'] > exps['pb_exact']] = 'A'
    s[exps['pb_exact'] > exps['pa_exact']] = 'B'
    res = {
        'comparison_type': 'exact',
        'selected': s
    }
    return res

def compare_experiments(exps, method_name, method_fn):
    s = np.empty(k_experiments, dtype=object)
    n = exps['n_points']
    for i, (sa, sb) in enumerate(zip(exps['sa'], exps['sb'])):
        s[i] = method_fn(sa=sa, na=n, sb=sb, nb=n)
    res = {
        'comparison_type': method_name,
        'selected': s
    }
    return res

def method_guesses(method, exact, n_points):
    e = exact['selected']
    m = method['selected']
    s = {
        'name': method['comparison_type'],
        'n_points': n_points,
        'experiments': len(e),
        'correct': np.sum(e == m),
        'not_sure': len(e) - np.count_nonzero(m),
        'incorrect': np.count_nonzero(m) - np.sum(e == m)
    }
    return s

Подсчет очков:

for each experiment:
    score_exact = N * max(pa, pb) 
    score_method = N * p_selected; p_selected = p_a if 'A', p_b if 'B', (pa + pb)/2 if None.

    score_method_accum += score_method / score_exact
    
Такой способ подсчета очков неоптимален для сомневающихся алгоритмов.  
Но не важно.

In [ ]:
def method_score(exps, method_selected):
    m_a = np.zeros_like(exps['pa_exact'])
    m_a[method_selected == 'A'] = 1
    m_a[method_selected == None] = 0.5 #todo: rewrite; should be elementwise is None
    m_b = np.zeros_like(exps['pb_exact'])
    m_b[method_selected == 'B'] = 1
    m_b[method_selected == None] = 0.5 #todo: rewrite should be elementwise is None
    selected = m_a * exps['pa_exact'] + m_b * exps['pb_exact']
    exact = np.maximum(exps['pa_exact'], exps['pb_exact'])
    score = np.divide(selected, exact)
    return np.sum(score) / len(score)

Сравнение 

In [ ]:
n_points = np.array([10, 100, 1000, 5000, 10000, 50000, 100000, 500000, 1000000])
#n_points = 100000
k_experiments = 100
pa_base = 0.1

comparisons = {
    'means': compare_groups_binomial,
    'bootstrap': compare_groups_bootstrap_binomial,
    'bayes': compare_groups_bayes_binomial,
}

df = pd.DataFrame(columns=['name', 'n_points', 'experiments', 
                           'correct', 'not_sure', 'incorrect', 'score'])

for n in n_points:
    exps = gen_binomial_experiments(n, k_experiments, pa_base)
    cmp_exact = compare_experiments_binomial_exact(exps)
    for k, v in comparisons.items():
        cmp = compare_experiments(exps, method_name=k, method_fn=v)
        stat = method_guesses(method=cmp, exact=cmp_exact, n_points=n)
        stat['score'] = method_score(exps, cmp['selected'])
        df = pd.concat([df, pd.DataFrame([stat])], ignore_index=True)

df.head(10)

In [ ]:
px.line(df, x='n_points', y='correct', color='name', markers='markers', log_x=True)

In [ ]:
px.line(df, x='n_points', y='score', color='name', markers='markers', log_x=True)

*При большом количестве точек решения бутстрапа и байесовского моделирования совпадают.*  
*Бета-распределение и биномиальное распределение переходят в нормальное *
$$
N(p, \frac{p(1-p)}{n}) ?
$$

Бета:  https://en.wikipedia.org/wiki/Beta_distribution#Special_and_limiting_cases  
Биномиальное: https://en.wikipedia.org/wiki/Binomial_distribution#Normal_approximation  

# Алгоритм оценки А/Б-тестов на основе проверки статистических гипотез

Общая идея:

Сформулировать нулевую гипотезу.
Выбрать уровень стат. значимости и мощности.

* Популяция
* Распределяем в 2 выборки
* Получаем 2 набора данных
* Считаем среднее и стандартное отклонение в каждом варианте
* считаем стандартную ошибку среднего для каждого варианта
* Предполагаем, что среднее по популяции каждого варианта находится вблизи среднего, посчитанного по выборке. 
* На основании центральной предельной теоремы сумма большого количества значений из одного распределения имеет нормальное распределение; среднее по выборке - сумма большого количества величин из одинакового распределения. На этом основании объявляем, что реальное среднее для каждого из вариантов распределено нормально вокруг среднего из выборки со стандартной ошибкой среднего [как это согласуется с частотной интерпретацей?] 
* Переходят к распределению для разницы двух величин: средние вычитаются, стандартные отклонения sqrt(s1^2 + s2^2).
* считают p-значение. Проверяют, достаточно ли оно мало для объявления стат. значимости.


Напоминалка:
https://towardsdatascience.com/the-math-behind-a-b-testing-with-example-code-part-1-of-2-7be752e1d06f


Вопросы: 2 категории - концептуальные и механика применения.

* Как интерпретировать доверительный интервал и стат. значимость?
* Как считается мощность? Особенно если распредления не симметричные?
* Вариантов больше 2 (групповые поправки)
* Подглядывания (и гарантии на значимость и мощность в этом случае)
* Оценка размера выборки если дисперсия не связана со средним?

### Статистические тесты, доверительные интервалы, проверка статистических гипотез.

**Статистические тесты**

Картинки про типы тестов:  
https://blog.statsols.com/types-of-statistical-tests  

https://duckduckgo.com/?q=how+to+choose+statistical+test&t=ffab&iar=images&iax=images&ia=images  

https://www.google.com/search?q=how+to+choose+statistical+test&source=lnms&tbm=isch&sa=X&ved=2ahUKEwip1s-rjuP3AhWIr4sKHY2HCHQQ_AUoAXoECAEQAw&biw=1280&bih=603&dpr=1.5 

**Доверительный интервал**

**Проверка статистических гипотез**

Проверка гипотез: https://en.wikipedia.org/wiki/Statistical_hypothesis_testing 


### t-тест и центральная предельная теорема

**Откуда берется t-тест**  

См. https://en.wikipedia.org/wiki/Student%27s_t-test и https://en.wikipedia.org/wiki/Student's_t-distribution .  

$$
\frac{X - \mu}{S/\sqrt{n}}
$$
X - sample mean, S - Bessel-corrected sample variance.

При больших N t-распределение почти совпадает с нормальным.

t-критерий:

$P(\frac{X - \mu}{S/\sqrt{n}} | \mu, \sigma) = t(...)$

**Центральная предельная теорема**



<center>
<img src="central_limit_theorem_and_t_test.png" alt="central_limit_theorem_and_t_test" width="600"/>
</center>

**Используемые предположения**

**Какой вариант лучше?**

Нужно $P(p_A > p_B)$.

Статистическая гипотеза: $p_A = p_B$.  
p-значение: $P(data | p_A = p_B) = p$

Дальше обычно говорят, что если $p$ мало, то $p_A \ne p_B$.

Как связано p-значение с нужной вероятностью $P(p_A > p_B)$?

Сложности:  
Из того, что $P(data | p_A = p_B)$ мало не следует, что $p_A \ne p_B$.  
Чтобы перейти от $P(data | p_A = p_B)$ к $P(p_A = p_B | data)$ нужно соотношение Байеса.  
Но дальше все равно непонятно, как $P(p_A = p_B | data)$ соотносится с $P(p_A > p_B)$.

**Каковы оценки метрики в каждой группе?**

Нужны оценки плотности вероятности P(p_A), P(p_B).

Есть средние в выборках.  

Для оценки неопределенности предлагается считать доверительные интервалы.  
Считаются в предположении, что выполняется центральная предельная теорема.  
$$
P(p_A) = N(p_{A, mean}, \sigma_A)
\\
P(p_B) = N(p_{B, mean}, \sigma_B)
$$
Доверительные интервалы дают оценки:  
$$
P(p_A \in [p_{A, mean} - 2\sigma_A, p_{A, mean} + 2\sigma_A]) = \alpha
\\
P(p_B \in [p_{B, mean} - 2\sigma_B, p_{B, mean} + 2\sigma_B]) = \alpha
$$


Кажется, что такие оценки занижают неопределенность.  
Не обязательно, что среднее равно именно среднему в выборке, может быть в окрестностях.  

Нужно смотреть модель

$$
P(mu, sigma | data) \\
P(data | mu, sigma) = N(mu, sigma) \\
mu - \mbox{нормальное распределение вокруг среднего значения в выборке} \\
sigma - \mbox{широкое нормальное распределение с центром на стандартной ошибке среднего для выборки}
$$

Или сравнить с бета-распределением.

Можно проверить, посэмлировав распределение.   
Если нормальное приближение делает завышенную по уверенности оценку, то при повторном применении этого приближения
не получится заявленная вероятность попадания среднего в доверительный интервал.

Фиксируется p.  
Сэмплируется.  
Считается N(mu, sigma).  
Проверяется попадание попадание p в определенный доверительный интервал.  


Нормальное или t-распределение можно рассматривать как оценку распределения P(mu | data).  
При этом приходится отказаться от претензий на установление реального среднего.  
И иметь дело с теми же проблемами, что и для любых других моделей - насколько адекватны предположения,
насколько адекватны прогнозы.  
Одна из проблем - распределение не ограничено интервалом [0,1] и есть ненулевая плотность вероятности за границами области.

t-критерий:

$P(\frac{X - \mu}{S/\sqrt{n}} | \mu, \sigma) = t(...)$

Нужно

$P(\mu, \sigma | X)$

Скорее всего можно предположить фиксированное $\sigma$.

Дальше

$$
P(\mu, \sigma | X) = \frac{P(X| \mu, \sigma) P(\mu, \sigma)}{\int d\mu P(X| \mu, \sigma) P(\mu, \sigma)}
$$

Если приближенно считать функцию правдоподобия нормальной,
можно задать сопряженное априорное распределение.  
https://en.wikipedia.org/wiki/Conjugate_prior#Continuous_distributions  
Сопряженное априорное также нормальное.  
Только не очень понятно, как выбирать параметры -- чтобы не на основе сэмпла.

In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

prior_conv = 0.3

a_total = 100
a_conv = a_total * prior_conv

#x = np.linspace(0, 1, 1001)
x = np.linspace(0, a_total, 10000)
fig = go.Figure()

# Norm
mu = a_conv
var = a_total * prior_conv * (1 - prior_conv)
sigma = np.sqrt(var)
stderrmean = sigma / np.sqrt(a_total)
print(mu, stderrmean)

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu, scale=stderrmean), mode='lines',
                         name=f"<br>total = {a_total}, conv = {a_conv}<br>mu={mu}, stderrmean={stderrmean}"))


#Uniform prior
# beta_prior = 1
# alpha_prior = 1

# alpha_post = alpha_prior + a_conv
# beta_post = beta_prior + (a_total - a_conv)
# fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, alpha_post, beta_post), mode='lines',
#                          name=f"<br>A: total = {a_total}, conv = {a_conv}<br>alpha_prior={alpha_prior}, beta_prior={beta_prior},<br>alpha_post={alpha_post}, beta_post={beta_post}"))



fig.update_layout(title='Posterior and Prior Distributions',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()

In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

prior_conv = 0.4

a_total = 10
a_conv = a_total * prior_conv

x = np.linspace(0, 1, 1001)
fig = go.Figure()

# Norm
## mu = 1 * a_conv + 0 * (a_total - a_conv)
## mu_c = 1 * (a_conv / a_total) + 0 * (a_total - a_conv) / a_total = a_conv / a_total
## std_c = sqrt( ( (1 - mu_c)^2 * a_conv + (0 - mu_c)^2 * (a_total - a_conv) ) / a_total )
##       = sqrt( (1 - mu_c)^2 mu_c + mu_c^2 (1 - mu_c) )
##       = sqrt( mu_c (1 - mu_c) )

mu_c = a_conv / a_total
sigma_c = np.sqrt(mu_c * (1 - mu_c))
stderrmean_c = sigma_c / np.sqrt(a_total)
print(mu_c, stderrmean_c)

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_c, scale=stderrmean_c), mode='lines',
                         name=f"<br>total = {a_total}, conv = {a_conv}<br>mu={mu}, stderrmean={stderrmean}"))


#Uniform prior
beta_prior = 1
alpha_prior = 1

alpha_post = alpha_prior + a_conv
beta_post = beta_prior + (a_total - a_conv)
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, alpha_post, beta_post), mode='lines',
                         name=f"<br>A: total = {a_total}, conv = {a_conv}<br>alpha_prior={alpha_prior}, beta_prior={beta_prior},<br>alpha_post={alpha_post}, beta_post={beta_post}"))



fig.update_layout(title='Posterior and Prior Distributions',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()

**Насколько один вариант лучше другого?**

Нужно $P(p_A/p_B)$ или $P(p_A - p_B)$.  
Распределение $P(p_A/p_B)$ обычно не считают - ограничиваются отношением средних в выборках $p_A/p_B$.  

Cчитают $P(p_A - p_B)$.  
С учетом предположений о нормальном распределении $p_A, p_B$
$$
P(p_A - p_B) = N(p_A - p_B; s_{AB}) 
\\
s_{AB} = \sqrt{s_A^2 / N_1 + s_B^2 / N_2} * \sqrt{N_1 + N_2} \quad \mbox{(или что-то вроде)}
$$
(См. https://mathworld.wolfram.com/NormalDifferenceDistribution.html, https://math.stackexchange.com/questions/917276/distribution-of-the-difference-of-two-normal-random-variables, https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables, https://en.wikipedia.org/wiki/Normal_distribution#Operations_on_normal_deviates)

При этом предположения о нормальном распределении $p_A, p_B$ излишне оптимистичны.  
Поэтому оценка разности также будет занижена.  

Можно попробовать проверить сэмплированием.

In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

prior_conv = 0.4

a_total = 1000
a_conv = a_total * prior_conv

b_total = 1000
b_conv = b_total * prior_conv * 1.1

x = np.linspace(-1, 1, 1001)
fig = go.Figure()

# Norm
mu_a = a_conv / a_total 
stderrmean_a = np.sqrt(mu_a * (1 - mu_a)) / np.sqrt(a_total)
print(mu_a, stderrmean_a)

mu_b = b_conv / b_total 
stderrmean_b = np.sqrt(mu_b * (1 - mu_b)) / np.sqrt(b_total)
print(mu_b, stderrmean_b)

mu_diff = mu_b - mu_a
stderrmean_diff = np.sqrt(stderrmean_a**2 + stderrmean_b**2)
print(mu_diff, stderrmean_diff)

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_diff, scale=stderrmean_diff), mode='lines',
                         name=f"Normal Diff"))


#Beta
beta_prior = 1
alpha_prior = 1
alpha_post_a = alpha_prior + a_conv
beta_post_a = beta_prior + (a_total - a_conv)
n_post_sample = 50000
post_sample_a = np.random.beta(alpha_post_a, beta_post_a, n_post_sample)
alpha_post_b = alpha_prior + b_conv
beta_post_b = beta_prior + (b_total - b_conv)
post_sample_b = np.random.beta(alpha_post_b, beta_post_b, n_post_sample)
post_sample_diff = post_sample_b - post_sample_a


fig.add_trace(go.Histogram(x=post_sample_diff, histnorm='probability density', 
                           name='B-A', marker_color='red',
                           opacity=0.6))


fig.update_layout(title='P(B-A)',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()

**Какой вариант лучше?**

Нужно $P(p_A > p_B)$.

Обычно пытаются проверять статистические гипотезы.  
Предполагают $p_A = p_B$ и считают, какова вероятность получить имеющиеся средние: $P(p_{mu,A}, p_{mu,B} | p_A = p_B)$.

Для t-тестов вместо статистических гипотез можно посчитать понятное значение $P(p_B > p_A)$ с помощью
кумулятивной функции распределения нормального распределения.

$$
P(p_A > p_B) = \mbox{norm.diff.cdf(x=0)}
\\
P(p_B > p_A) = 1 - \mbox{norm.diff.cdf(x=0)}.
$$

In [ ]:
x = np.linspace(-0.5, 0.5, 1001)
fig = go.Figure()
# fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_diff, scale=stderrmean_diff), mode='lines',
#                          name=f"Normal Diff"))
fig.add_trace(go.Scatter(x=x, y=stats.norm.cdf(x, loc=mu_diff, scale=stderrmean_diff), mode='lines',
                         name=f"Normal Diff Accumulated"))
fig.update_layout(title='P(B-A) Accumulated',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  xaxis_range=[-0.1, 0.1],
                  height=550)

fig.show()


pa_gt_pb_norm = stats.norm.cdf(x=0, loc=mu_diff, scale=stderrmean_diff)
pb_gt_pa_norm = 1 - pa_gt_pb_norm
print(f"P(pa > pb) = {pa_gt_pb_norm}, P(pb > pa) = {pb_gt_pa_norm}")

#diff = B - A
pb_gt_pa_diff = len(post_sample_diff[post_sample_diff > 0]) / len(post_sample_diff)
pa_gt_pb_diff = 1 - pb_gt_pa_diff
print(f"P(pa > pb) = {pa_gt_pb_diff}, P(pb > pa) = {pb_gt_pa_diff}")

**Проверка статистической гипотезы $p_A = p_B$**

Во-первых, вопрос о $p_A = p_B$ - это не тот вопрос, на который нужно отвечать.  
Нужно $P(p_A > p_B)$ -- см. выше.  

Во-вторых, способ проверки гипотезы вызывает вопросы.  
Предполагают $p_A = p_B$ и считают, какова вероятность получить имеющиеся средние: $P(p_{mu,A}, p_{mu,B} | p_A = p_B)$.

Выбирают уровень значимости $\alpha$.  
Обычно $\alpha = 0.95$.  
Находят симметричный относительно центра интервал, внутри которого лежит $\alpha$ плотности вероятности.  
Для $\alpha = 0.95$ это [mu_diff - 2 s_diff, mu_diff + 2 s_diff].  
По распределению разности проверяют, лежит ли точка 0 в интервале [mu_diff - 2 s_diff, mu_diff + 2 s_diff].  
Если попадает, то гипотеза не отвергается.  
Если не попадает, то отвергается.  

**Сколько должен продолжаться эксперимент?**  

Для конверсий дисперсия связана со средним.  
Можно задаться величиной эффекта.  
Оценить, как будут выглядеть распределения средних.  
На этом основании оценить, сколько нужно наблюдений для детектирования эффекта с определенной вероятностью.  


Если дисперсия заранее неизвестна, то возникают проблемы с оценкой длительности.  
Не ясно, чему равна стандартная ошибка среднего.


Еще есть проблема подглядывания.  
Оценку длительности предлагается фиксировать до начала эксперимента.  

**Как работают оценки длительности?**

Один из калькуляторов: https://www.evanmiller.org/ab-testing/sample-size.html .



In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

prior_conv = 0.4

a_total = 1000
a_conv = a_total * prior_conv

b_total = a_total
b_conv = b_total * prior_conv * 1.05

x = np.linspace(0, 1, 1001)
fig = go.Figure()

# Norm
## mu = 1 * a_conv + 0 * (a_total - a_conv)
## mu_c = 1 * (a_conv / a_total) + 0 * (a_total - a_conv) / a_total = a_conv / a_total
## std_c = sqrt( ( (1 - mu_c)^2 * a_conv + (0 - mu_c)^2 * (a_total - a_conv) ) / a_total )
##       = sqrt( (1 - mu_c)^2 mu_c + mu_c^2 (1 - mu_c) )
##       = sqrt( mu_c (1 - mu_c) )

mu_ca = a_conv / a_total
sigma_ca = np.sqrt(mu_ca * (1 - mu_ca))
stderrmean_ca = sigma_ca / np.sqrt(a_total)
print(mu_ca, stderrmean_ca)

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_ca, scale=stderrmean_ca), mode='lines',
                         name=f"<br>total = {a_total}, conv = {a_conv}<br>mu={mu_ca}, stderrmean={stderrmean_ca}"))

mu_cb = b_conv / b_total
sigma_cb = np.sqrt(mu_cb * (1 - mu_cb))
stderrmean_cb = sigma_cb / np.sqrt(b_total)
print(mu_cb, stderrmean_cb)

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_cb, scale=stderrmean_cb), mode='lines',
                         name=f"<br>total = {b_total}, conv = {b_conv}<br>mu={mu_cb}, stderrmean={stderrmean_cb}"))

fig.add_vline(x=mu_cb)

fig.update_layout(title='Posterior and Prior Distributions',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()

pval = 1 - stats.norm.cdf(x=mu_cb, loc=mu_ca, scale=stderrmean_ca)
print(pval)


def pval(prior_conv, exp_effect, N):
    a_total = N
    a_conv = a_total * prior_conv
    b_total = a_total
    b_conv = b_total * prior_conv * exp_effect
    mu_ca = a_conv / a_total
    sigma_ca = np.sqrt(mu_ca * (1 - mu_ca))
    stderrmean_ca = sigma_ca / np.sqrt(a_total)
    mu_cb = b_conv / b_total
    pval = 1 - stats.norm.cdf(x=mu_cb, loc=mu_ca, scale=stderrmean_ca)
    return pval


N = np.linspace(100, 10000)
y = np.array([pval(prior_conv, 1.05, Ni) for Ni in N])

fig = go.Figure()
fig.add_trace(go.Scatter(x=N, y=y, mode='lines'))
fig.add_hline(y=0.05)
fig.update_layout(title='p-val',
                  xaxis_title='N',
                  yaxis_title='p-val',
                  hovermode="x",
                  height=550)

In [ ]:
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go

prior_conv = 0.4

a_total = 100
a_conv = a_total * prior_conv

b_total = 100
b_conv = b_total * prior_conv * 1.1

x = np.linspace(-1, 1, 1001)
fig = go.Figure()

# Norm
mu_a = a_conv / a_total 
stderrmean_a = np.sqrt(mu_a * (1 - mu_a)) / np.sqrt(a_total)
print(mu_a, stderrmean_a)

mu_b = b_conv / b_total 
stderrmean_b = np.sqrt(mu_b * (1 - mu_b)) / np.sqrt(b_total)
print(mu_b, stderrmean_b)

mu_diff = mu_b - mu_a
stderrmean_diff = np.sqrt(stderrmean_a**2 + stderrmean_b**2)
print(mu_diff, stderrmean_diff)

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_diff, scale=stderrmean_diff), mode='lines',
                         name=f"100"))


a_total = 500
a_conv = a_total * prior_conv

b_total = 500
b_conv = b_total * prior_conv * 1.1

# Norm
mu_a = a_conv / a_total 
stderrmean_a = np.sqrt(mu_a * (1 - mu_a)) / np.sqrt(a_total)
print(mu_a, stderrmean_a)

mu_b = b_conv / b_total 
stderrmean_b = np.sqrt(mu_b * (1 - mu_b)) / np.sqrt(b_total)
print(mu_b, stderrmean_b)

mu_diff = mu_b - mu_a
stderrmean_diff = np.sqrt(stderrmean_a**2 + stderrmean_b**2)
print(mu_diff, stderrmean_diff)

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_diff, scale=stderrmean_diff), mode='lines',
                         name=f"{a_total}"))
fig.add_vline(x=0)
fig.update_layout(title='P(B-A)',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()


def pval(prior_conv, exp_effect, N):
    a_total = N
    a_conv = a_total * prior_conv
    b_total = a_total
    b_conv = b_total * prior_conv * exp_effect
    mu_a = a_conv / a_total 
    stderrmean_a = np.sqrt(mu_a * (1 - mu_a)) / np.sqrt(a_total)
    mu_b = b_conv / b_total 
    stderrmean_b = np.sqrt(mu_b * (1 - mu_b)) / np.sqrt(b_total)
    mu_diff = mu_b - mu_a
    stderrmean_diff = np.sqrt(stderrmean_a**2 + stderrmean_b**2)
    if mu_diff > 0:
        pval = stats.norm.cdf(x=0, loc=mu_diff, scale=stderrmean_diff)
    else:
        pval = None
    return pval

N = np.linspace(100, 10000)
y = np.array([pval(prior_conv, 1.05, Ni) for Ni in N])

fig = go.Figure()
fig.add_trace(go.Scatter(x=N, y=y, mode='lines'))
fig.add_hline(y=0.05)
fig.update_layout(title='p-val',
                  xaxis_title='N',
                  yaxis_title='p-val',
                  hovermode="x",
                  height=550)

# Ссылки

Посмотреть:

J. Kruschke  
https://www.medicine.mcgill.ca/epidemiology/Joseph/courses/EPIB-682/Kruschke2013.pdf  
https://www.youtube.com/watch?v=fhw1j1Ru2i0